# 多層パーセプトロン

このノートブックは、以下のページに基づいて作成されています。

- http://gluon.mxnet.io/chapter03_deep-neural-networks/mlp-scratch.html

In [1]:
require 'mxnet'

true

In [2]:
@data_ctx = MXNet.cpu
@model_ctx = MXNet.cpu
#@model_ctx = MXNet.gpu

#<MXNet::Context:0x00007fa718b848b0 @device_type_id=1, @device_id=0>

## データのロード

必要ならダウンロードする

In [3]:
unless File.exist?('train-images-idx3-ubyte') &&
       File.exist?('train-labels-idx1-ubyte')
  system("wget http://data.mxnet.io/mxnet/data/mnist.zip")
  system("unzip -x mnist.zip")
end

In [4]:
num_inputs = 784
num_outputs = 10
batch_size = 64
num_examples = 60000

train_iter = MXNet::IO::MNISTIter.new(
  batch_size: batch_size,
  shuffle: true)
test_iter = MXNet::IO::MNISTIter.new(
  image: 't10k-images-idx3-ubyte',
  label: 't10k-labels-idx1-ubyte',
  batch_size: batch_size,
  shuffle: false)
nil

## パラメータ

ニューラルネットワークのパラメータを定義する。

In [5]:
#######################
#  Set some constants so it's easy to modify the network later
#######################
num_hidden = 256
weight_scale = 0.01

#######################
#  Allocate parameters for the first hidden layer
#######################
@w1 = MXNet::NDArray.random_normal(shape: [num_inputs, num_hidden], scale: weight_scale, ctx: @model_ctx)
@b1 = MXNet::NDArray.random_normal(shape: [num_hidden], scale: weight_scale, ctx: @model_ctx)

#######################
#  Allocate parameters for the second hidden layer
#######################
@w2 = MXNet::NDArray.random_normal(shape: [num_hidden, num_hidden], scale: weight_scale, ctx: @model_ctx)
@b2 = MXNet::NDArray.random_normal(shape: [num_hidden], scale: weight_scale, ctx: @model_ctx)

#######################
#  Allocate parameters for the output layer
#######################
@w3 = MXNet::NDArray.random_normal(shape: [num_hidden, num_outputs], scale: weight_scale, ctx: @model_ctx)
@b3 = MXNet::NDArray.random_normal(shape: [num_outputs], scale: weight_scale, ctx: @model_ctx)

@params = [@w1, @b1, @w2, @b2, @w3, @b3]
nil

In [6]:
@params.each do |param|
  param.attach_grad
end
nil

## 活性化関数

In [7]:
def relu(x)
  MXNet::NDArray.maximum(x, MXNet::NDArray.zeros_like(x))
end

:relu

## Softmax 出力

In [8]:
def softmax(y_linear)
  exp = MXNet::NDArray.exp(y_linear - MXNet::NDArray.max(y_linear))
  partition = MXNet::NDArray.nansum(exp, axis: 0, exclude: true).reshape([-1, 1])
  return exp / partition
end

:softmax

## Softmax 交差エントロピー損失関数

In [9]:
def cross_entropy(y_hat, y)
  return -MXNet::NDArray.nansum(y * MXNet::NDArray.log(y_hat), axis: 0, exclude: true)
end

:cross_entropy

In [10]:
def softmax_cross_entropy(y_hat_linear, y)
  return -MXNet::NDArray.nansum(y * MXNet::NDArray.log_softmax(y_hat_linear), axis: 0, exclude: true)
end

:softmax_cross_entropy

## モデル定義

In [11]:
def net(x)
  # first hidden layer
  h1_linear = MXNet::NDArray.dot(x, @w1) + @b1
  h1 = relu(h1_linear)

  # second hidden layer
  h2_linear = MXNet::NDArray.dot(h1, @w2) + @b2
  h2 = relu(h2_linear)

  # output layer
  y_hat_linear = MXNet::NDArray.dot(h2, @w3) + @b3
  return y_hat_linear
end

:net

## オプティマイザ

In [12]:
def sgd(params, lr)
  params.each do |param|
    param[0..-1] = param - lr * param.grad
  end
end

:sgd

## 評価尺度

In [13]:
def evaluate_accuracy(data_iter)
  numerator = 0.0
  denominator = 0.0
  data_iter.each_with_index do |batch, i|
    data = batch.data[0].as_in_context(@model_ctx).reshape([-1, 784])
    label = batch.label[0].as_in_context(@model_ctx)
    output = net(data)
    predictions = MXNet::NDArray.argmax(output, axis: 1)
    numerator += MXNet::NDArray.sum(predictions == label)
    denominator += data.shape[0]
  end
  return (numerator / denominator).as_scalar
end

:evaluate_accuracy

In [17]:
epochs = 10
learning_rate = 0.001
smoothing_constant = 0.01

epochs.times do |e|
  start = Time.now
  cumulative_loss = 0.0
  train_iter.each_with_index do |batch, i|
    data = batch.data[0].as_in_context(@model_ctx).reshape([-1, 784])
    label = batch.label[0].as_in_context(@model_ctx)
    label_one_hot = MXNet::NDArray.one_hot(label, depth: 10)
    loss = MXNet::Autograd.record do
      output = net(data)
      softmax_cross_entropy(output, label_one_hot)
    end
    loss.backward()
    sgd(@params, learning_rate)
    cumulative_loss += MXNet::NDArray.sum(loss).as_scalar
  end
  
  test_accuracy = evaluate_accuracy(test_iter)
  train_accuracy = evaluate_accuracy(train_iter)
  duration = Time.now - start
  puts "Epoch #{e}. Loss: #{cumulative_loss/num_examples}, Train_acc #{train_accuracy}, Test_acc #{test_accuracy} (#{duration} sec)"
end

Epoch 0. Loss: 0.049135282553980746, Train_acc 0.9857757687568665, Test_acc 0.9718000292778015 (3.999875 sec)
Epoch 1. Loss: 0.042806731517116225, Train_acc 0.9872098565101624, Test_acc 0.9722999930381775 (3.910006 sec)
Epoch 2. Loss: 0.03742000448070466, Train_acc 0.987993597984314, Test_acc 0.9729999899864197 (3.895349 sec)
Epoch 3. Loss: 0.03264522926310698, Train_acc 0.9892275929450989, Test_acc 0.9731000065803528 (4.054005 sec)
Epoch 4. Loss: 0.02842021626768013, Train_acc 0.9905449748039246, Test_acc 0.9740999937057495 (4.068799 sec)
Epoch 5. Loss: 0.024795985047953824, Train_acc 0.9917622804641724, Test_acc 0.9750999808311462 (4.173076 sec)
Epoch 6. Loss: 0.021673260604652266, Train_acc 0.9926294088363647, Test_acc 0.975600004196167 (4.107134 sec)
Epoch 7. Loss: 0.018886250545146564, Train_acc 0.9934131503105164, Test_acc 0.9757999777793884 (4.093758 sec)
Epoch 8. Loss: 0.016440149055824926, Train_acc 0.994396984577179, Test_acc 0.9768000245094299 (4.08556 sec)
Epoch 9. Loss: 0.

10

## モデルを予測で使用する

In [15]:
require 'chunky_png'
require 'base64'

def imshow(ary)
  height, width = ary.shape
  fig = ChunkyPNG::Image.new(width, height, ChunkyPNG::Color::TRANSPARENT)
  ary = ((ary - ary.min) / ary.max) * 255
  0.upto(height - 1) do |i|
    0.upto(width - 1) do |j|
      v = ary[i, j].round
      fig[j, i] = ChunkyPNG::Color.rgba(v, v, v, 255)
    end
  end

  src = 'data:image/png;base64,' + Base64.strict_encode64(fig.to_blob)
  IRuby.display "<img src='#{src}' width='#{width*2}' height='#{height*2}' />", mime: 'text/html'
end

:imshow

In [16]:
# Define the funtion to do prediction
def model_predict(data)
  output = net(data)
  MXNet::NDArray.argmax(output, axis: 1)
end

samples = 10
sample_iter = test_iter = MXNet::IO::MNISTIter.new(
  image: 't10k-images-idx3-ubyte',
  label: 't10k-labels-idx1-ubyte',
  batch_size: samples,
  shuffle: true)
sample_iter.each do |batch|
  batch = sample_iter.next_batch
  data = batch.data[0].as_in_context(@model_ctx)
  label = batch.label[0]

  im = data.transpose(axes: [1, 0, 2, 3]).reshape([10*28, 28, 1])
  imshow(im[0..-1, 0..-1, 0].to_narray)

  pred = model_predict(data.reshape([-1, 784]))
  puts "model predictions are: #{pred.inspect}"
  puts
  puts "true labels: #{label.inspect}"
  break
end

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAEYCAAAAABqgMnqAAAIc0lEQVR4nO3ZeVQURxoA8K8H5IqoxAiKgUhiBFwEQQXFAUZXgyHKoYIaEAYDkSMQJYiwQmYgChEUQSSKRhRFYMNlQAJeYRBRNCuICaDiKsEEIiioGDmU6fQMMNRMV/He/mU2u997dBf9e11VX72prqkeDowSnFeMY1TxqO+YkRGcX5yRsd+MCXk8fanA3X2Pw2J394+vMVEwBcUlkyXHrszHDZn3mILTt1MQ3A95Prq6hnxDa6+51qUAFiYIfqK89mj7g0549Aye1J2mOAP9+FTU9yXRYlEFFlX3egP0x+MH4RpjsPE8Dr1yjAEaph/DjND0b9JdOVRdeDOwMbh0JU3/Grf8O2Bj6B4D5siNbAMW6sZuZ453udIq1bNqaw8haBKmDLBrbjVohx5ubXabpbQDrZaiOA85kd10206+jnp7rmkzgt09tPiNkBB1MU13fj1fd51cm5eX1EnP/U2HFm1sUOxt9TJze5c93Dm2/j9hUum4cb54S3VDBzIqr/pz+4pQ1W+Tn6PQT5WFM5wD8upSd6QWRn1lLodKCwoKavP3ce/vSnV339V1bgaCWjsvOtvmetvrL90alpOzdZ36BwimhNR46fGPnR2cBdz9FDKReI7uC4/3DLetf9Dg15IRVEnO7pf1K+SSYd9X6FypQbLxmAJH45DenimUkSs9++cIf+wgBLcK6eeb43EjpH49Qceo1v1bwKBWtalyzxdzRgxBqzuzqJN6QrQVGWrmTqCjXLsAix9NFZ/YLgY8dkGHp+zqnDB5bKMSBgu60/zOft8lj7dbnOJdzLbVtNXXpep9cEgem91nf5ZfE2Omo/4w0OGiYipVs+OPN5w6HhPwzoG7rDyhOcJrlhNfmI7pLS7+qsgPC6sUr8Chhfedg3Fx1nTOWyzUry/72kAJaptBzUQRl+UaTYTaTJ4DM4PSjRUwfC50rrXxqmxnFqQ3NBUwMSDAPlc2lRSwKC1tcL4sGy3PRaOghiHAd9cJOP89gBv9eNQIBTi1nVCt0J5ZrXrwaOQGMBBL6BBfDwbe78bju8yC8fI8IZW8NwF2Ah7XMDleSSPg5DHMo+w3AjLRcgUIePFpk+AuCa9pGR0DErLjlaJ5p1gabQEYtBhPS0N7b+oo1VL2bDyRP1waZ8TCXq+E9PR0yaowMZBdbU+4r69vqOSbiTOhzds+2FSG4kLFKDhtNkATBucvZiJ6PMBxRbQKPn/uLBMrMW3qlO3hqcuq1pZD7ZJxyA2ReXLIN5fr1d9mrEVw1fDlvqrvmeOEq40IDn/o+sps1/xLUqhD8NLvkmN/9ZqV0Dn8NUqGKXbt3d2XXRcWg9xsGUqldsqECdxTsqsDcojEw0R4/j4RtyhpXiUhEv/lmHL5UxKqlftbxU4ioKMd5/YlJQLawNVbiwmf+JlupWtabyV7YVF1UtPP/kVjrPHV9u4FjeW95/D44mlQ2czkXDxqtidz4Qmht0zQSQewOCYcoLAq5DEOgy649hRl2o5cGEHfo4lWjyGsEDBoucmzd/U61amAwaxzxhe4Be0cLRy6qoc6MBOkrx6HcDnxOfNP3y0cZr/pZQkuO1sBh5712ypb0my/wCKsCD1xK9UnG49QtGGpIBMIiIk/CWp40z8SUXtH/3USWhXp5K3Ho4bZsXd3f0RoU7d8+umwXjzqND2Ll00VBTSueeocAXi0+mffKnSPgaCGh2iS8BzgMSNDJVF+RR9B09Uv4xMAj8pp9OEIIOBiy0fSxUbVaTobD8Pnl5lHv1FGQe0RRXzrN+gDmFfcwEn/t2eIAuqbUk1gXrEw1M03mFqogJUP6L0zU1QFiRyLrJ4vFdvcCWY/LoA+J8EPY11+UMQDsZIJHVcY+cvqM6zeDkStSE/vhcrl5iMjiAxfla/va0p2pZ2YQcDFXxU1/UUxJwi4oTqF+48mLE7MTDKEJ3LPjBFMXqvx5MupO+7gcJ4jJJtE9sogko/gFA1Ikr39UdcPjlyPYAza7aS7icqfIxg6QipHvX/P0r2CYOdjEAzuYmYkeDwL9mx/iWBNGfD3SQq2FwOhMkMhlU8oymNgs7+ooj8tja+YZ7dZWxOdkMIVewcGdiniwE+fBkkLsjVHbmxtS6Wn7JlstL63safE2CATphWx0CZH70WEU9P9XQBaLEzRhU2ZdjD5Q6QdtM3dzSWdN8OgfTULt9fD2LFq48bCI345C/MsA/d3noHs7FWySYZU25cWpO2gvH59Jb5NVvyv4vH2OWQ0muhDxGUW6AUFdKbFN0m4ciPFoQhonCGmxYYEbDxJUVlRpDudaXrzQwJqaFAnEVO4k6ZjgYRcquc5CY1d6MabJLSZRHkCCY1oupGEr9lTh4CELoYdZLSh7tcQkaYbgIgUFUdGuqCRjH6uQEZ2/KmRcj1E317/uhYWU3M2iN852tGMQ52hzYbaMgyGqg2eld9m4zT+cEnxVRcTC16Xnm5WwQo2SndGL48sDRePm8lCPwBxvbFPqwo8Zf1AAHGNhz80ZXbGSzgl7GpLTD6W7HD+/hnUs3Eotqr0FLLx7aT6vV7b6m3gYBsLw64FGQWmxxipgGwMZGgdK9v6ryiepyyHkzIoeHZ/SB2q82ehOJ6pSyB5ut+pY9ZNWL5bFUE35m83s96UzrewF0kSWoXgBenxxkGPLhC9F/ULwBUEpW+4arf4S5bHgVjLSNsWBO/5tDSusxn+3vYgruoFggNHDEy+Ib2AxMf/UR6F5XS5UMjDoZAW8IAnEAh4GBQMnXnlQhZKr4jIbS6iFlFUNIAdDiX38RjhsVAkYh6pdHk5D61chtHS3sgO8tWKFHsj12Z0dLQoOppZIKMxKBIKFwmFDAkwiIn/DIWj3ikayYWN0aOhaGRw2SiECjKO2qFRUUgJh4t/AEAF408DPIXzAAAAAElFTkSuQmCC' width='56' height='560' />"

model predictions are: 
[2, 2, 7, 2, 8, 6, 8, 1, 2, 9]
<MXNet::NDArray 10 @cpu(0)>

true labels: 
[2, 2, 7, 2, 8, 6, 8, 1, 2, 9]
<MXNet::NDArray 10 @cpu(0)>
